### Dataframes 
Dataframes are a special type of RDDs. They are similar to, but not the same as, pandas dataframes. They are used to store two dimensional data, similar to the type of data stored in a spreadsheet. Each column in a dataframe can have a different type and each row contains a `record`.

Spark DataFrames are similar to `pandas` DataFrames. With the important difference that spark DataFrames are **distributed** data structures, based on RDDs.

In [1]:
import os
import sys

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType

In [2]:
# Just like using Spark requires having a SparkContext, using SQL requires an SQLContext
sqlContext = SQLContext(sc)
sqlContext

In [3]:
# One way to create a DataFrame is to first define an RDD from a list of rows
some_rdd = sc.parallelize([Row(name=u"John", age=19),
                           Row(name=u"Smith", age=23),
                           Row(name=u"Sarah", age=18)])
some_rdd.collect()

[Row(age=19, name=u'John'),
 Row(age=23, name=u'Smith'),
 Row(age=18, name=u'Sarah')]

In [4]:
# The DataFrame is created from the RDD or Rows
# Infer schema from the first row, create a DataFrame and print the schema
some_df = sqlContext.createDataFrame(some_rdd)
some_df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [5]:
# A dataframe is an RDD of rows plus information on the schema.
# In our case, the content of the RDD is the same as the content of the dataframe. 
print type(some_rdd),type(some_df)
print 'some_df =',some_df.collect()
print 'some_rdd=',some_rdd.collect()

<class 'pyspark.rdd.RDD'> <class 'pyspark.sql.dataframe.DataFrame'>
some_df = [Row(age=19, name=u'John'), Row(age=23, name=u'Smith'), Row(age=18, name=u'Sarah')]
some_rdd= [Row(age=19, name=u'John'), Row(age=23, name=u'Smith'), Row(age=18, name=u'Sarah')]


### Example of using an RDD

In [6]:
# In this case we create the dataframe from an RDD of tuples (rather than Rows) and provide the schema explicitly
another_rdd = sc.parallelize([("John", 19), ("Smith", 23), ("Sarah", 18)])
# Schema with two fields - person_name and person_age
schema = StructType([StructField("person_name", StringType(), False),
                     StructField("person_age", IntegerType(), False)])

# Create a DataFrame by applying the schema to the RDD and print the schema
another_df = sqlContext.createDataFrame(another_rdd, schema)
another_df.printSchema()
# root
#  |-- age: binteger (nullable = true)
#  |-- name: string (nullable = true)

root
 |-- person_name: string (nullable = false)
 |-- person_age: integer (nullable = false)



## Loading dataframes from JSON files
[JSON](http://www.json.org/) is a very popular readable file format for storing structured data.
Among it's many uses are **twitter**, `javascript` communication packets, and many others. In fact this notebook file (with the extension `.ipynb` is in json format. JSON can also be used to store tabular data and can be easily loaded into a dataframe.

In [8]:
# when loading json files you can specify either a single file or a directory containing many json files.
path = "../Data/people.json"
!cat $path 

{"name":"Michael"}
{"name":"Andy", "age":30}
{"name":"Justin", "age":19}


In [9]:
# Create a DataFrame from the file(s) pointed to by path
people = sqlContext.read.json(path)
print 'people is a',type(people)
# The inferred schema can be visualized using the printSchema() method.
print
people.printSchema()
# root
#  |-- age: IntegerType
#  |-- name: StringType

people is a <class 'pyspark.sql.dataframe.DataFrame'>

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



### What formats does spark-sql support?

According to [this post](https://databricks.com/blog/2015/01/09/spark-sql-data-sources-api-unified-data-access-for-the-spark-platform.html), spark supports many formats. However, I could not find definite documentation on which 
formats are supported.

In terms of syntax, you can use the format 
```python
sqlContext.read.format('json').load('python/test_support/sql/people.json')
```
Where instead of `json` you can use **`parquet`,`text`** and suppposedly other formats, but I could not find an authoritative list of formats. It seems that `csv` is not supported at this time.

### Excercise
Write code to read a csv file in which the first row defines the names of the columns and turn it into a
DataFrame with the appropriate Schema.

In [29]:
RDD=sc.textFile('../Data/example.csv')
RDD.collect()

[u'Col1, Col2, Col3', u'11,12,13', u'21,22,23', u'31,32,33']

In [50]:
#Exercise
s_schemaCSVString = RDD.first()
schemaCSV = StructType([StructField(field_name, IntegerType(), True) for field_name in s_schemaCSVString.split(", ")])
l_CSVData = RDD.collect()
l_CSVData.pop(0)
print l_CSVData
tRDD_CSVData = sc.parallelize(l_CSVData).map(lambda x: x.split(",")).map(lambda x: map(int, x)).map(lambda x: tuple(x))
csvDF = sqlContext.createDataFrame(tRDD_CSVData, schema)
csvDF.printSchema()
csvDF.collect()

[u'11,12,13', u'21,22,23', u'31,32,33']
root
 |-- Col1: integer (nullable = true)
 |-- Col2: integer (nullable = true)
 |-- Col3: integer (nullable = true)



[Row(Col1=11, Col2=12, Col3=13),
 Row(Col1=21, Col2=22, Col3=23),
 Row(Col1=31, Col2=32, Col3=33)]

### Using HiveSQL queries on DataFrames
You can use [Hive `select` syntax](https://cwiki.apache.org/confluence/display/Hive/LanguageManual+Select) to select a subset of the rows in a dataframe.
Spark supports a [subset](https://spark.apache.org/docs/latest/sql-programming-guide.html#supported-hive-features) of the Hive SQL query language

To use sql on a dataframe you need to first `register` it as a `TempTable`.

In [52]:
people.show()
csvDF.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

+----+----+----+
|Col1|Col2|Col3|
+----+----+----+
|  11|  12|  13|
|  21|  22|  23|
|  31|  32|  33|
+----+----+----+



In [56]:
# Register this DataFrame as a table.
people.registerTempTable("people")
csvDF.registerTempTable("csvDF")
# SQL statements can be run by using the sql methods provided by sqlContext
teenagers = sqlContext.sql("SELECT name FROM people WHERE age >= 13 AND age <= 19")
for each in teenagers.collect():
    print(each[0])
    
test = sqlContext.sql("SELECT * FROM csvDF WHERE Col1 >= 13 AND Col2 <=40")
for each in test.collect():
    print(each)

Justin
Row(Col1=21, Col2=22, Col3=23)
Row(Col1=31, Col2=32, Col3=33)


## Using Hive and other RDBMS
Spark can interact efficiently with the Hive column-oriented RDBMS. That adds both the flexibility of using SQL and the benefit of performing SQL operations on disk, without necessarily loading all of the data to memory.

We will not use Hive in this course. However we will use the `Parquet` file format which allows **performing select operations before loading rows into memory. This is often a much faster approach than reading the whole file and then performing a `filter()` operation.**

### Parquet files
[Parquet](http://parquet.apache.org/) is a columnar format that is supported by many other data processing systems. Spark SQL provides support for both reading and writing Parquet files that automatically preserves the schema of the original data. 

In [66]:
dir='../Data'
parquet_file=dir+"/users.parquet"
!ls -ld $dir/*.parquet
!rm -rf ../Data/namesAndFavColors.parquet

drwxr-xr-x  7 wd  staff  238 Apr  6 16:47 ../Data/namesAndFavColors.parquet
-rw-r--r--  1 wd  staff  615 Apr  6 16:47 ../Data/users.parquet


In [67]:
#load a Parquet file
df = sqlContext.read.load(parquet_file)
df.show()

+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          null|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+



In [68]:
df2=df.select("name", "favorite_color")
df2.show()

+------+--------------+
|  name|favorite_color|
+------+--------------+
|Alyssa|          null|
|   Ben|           red|
+------+--------------+



In [69]:
df2.write.save(dir+"/namesAndFavColors.parquet")
!ls -ld $dir/*.parquet

drwxr-xr-x  10 wd  staff  340 Apr 21 15:01 ../Data/namesAndFavColors.parquet
-rw-r--r--   1 wd  staff  615 Apr  6 16:47 ../Data/users.parquet


### A somewhat bigger example
This file contains 1/1000 of the original file, which contains about 9 million lines and whose size is about 7GB.

In [73]:
%cd ../Data
import urllib
# If these commands don't work, you can download the files using your rowser
f=urllib.urlretrieve("https://drive.google.com/open?id=0B8IGBNGc5gVANWtlUElzd0JxV1E")
!curl  'https://drive.google.com/open?id=0B8IGBNGc5gVANWtlUElzd0JxV1E' > test.tgz
!ls -lrt 
#!tar xzvf  weather.tgz

/Users/wd/Workspace/spark/UCSD_BigData_2016/Data
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   261    0   261    0     0   1192      0 --:--:-- --:--:-- --:--:--  1197
total 42528
-rw-r--r--   1 wd  staff       615 Apr  6 16:47 users.parquet
-rw-r--r--   1 wd  staff   9600424 Apr  6 16:47 users-partition.pickle
-rw-r--r--   1 wd  staff       679 Apr  6 16:47 states.txt
-rw-r--r--   1 wd  staff        73 Apr  6 16:47 people.json
-rw-r--r--   1 wd  staff  10890659 Apr  6 16:47 hw2-input.txt
-rw-r--r--   1 wd  staff        22 Apr  6 16:47 hw2-files.txt
-rw-r--r--   1 wd  staff        43 Apr  6 16:47 example.csv
-rw-r--r--   1 wd  staff   1257260 Apr  6 16:47 Moby-Dick.txt
drwxr-xr-x   8 wd  staff       272 Apr 14 08:01 Weather
drwxr-xr-x  10 wd  staff       340 Apr 21 15:01 namesAndFavColors.parquet
-rw-r--r--   1 wd  staff       261 Apr 21 15:04 test.tgz


In [71]:
parquet_file=dir+"/Weather_sampled.parquet"
df = sqlContext.read.load(parquet_file)
print 'count=',df.count()
print 'columns=',df.columns[:5]

Py4JJavaError: An error occurred while calling o1165.load.
: java.lang.AssertionError: assertion failed: No predefined schema found, and no Parquet data files or summary files found under file:/Users/wd/Workspace/spark/UCSD_BigData_2016/Data/Weather_sampled.parquet.
	at scala.Predef$.assert(Predef.scala:179)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetRelation$MetadataCache.org$apache$spark$sql$execution$datasources$parquet$ParquetRelation$MetadataCache$$readSchema(ParquetRelation.scala:512)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetRelation$MetadataCache$$anonfun$12.apply(ParquetRelation.scala:421)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetRelation$MetadataCache$$anonfun$12.apply(ParquetRelation.scala:421)
	at scala.Option.orElse(Option.scala:257)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetRelation$MetadataCache.refresh(ParquetRelation.scala:421)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetR

#### Count the number of occurances of each measurement

In [74]:
L=df.groupBy('measurement').count().collect()
D=[(e.measurement,e['count']) for e in L]
sorted(D,key=lambda x:x[1], reverse=True)[:6]

AnalysisException: u"cannot resolve 'measurement' given input columns: [name, favorite_color, favorite_numbers];"

### Reading only select lines
Suppose we are only interested in snow measurements. We can apply an SQL query directly to the 
parquet files. As the data is organized in columnar structure, we can do the selection efficiently without loading the whole file to memory.

Here the file is small, but in real applications it can consist of hundreds of millions of records. In such cases loading the data first to memory and then filtering it is very wasteful.

In [66]:
query='SELECT station,measurement,year FROM parquet.`%s` WHERE measurement="SNOW"'%parquet_file
print query
df2 = sqlContext.sql(query)
print df2.count(),df2.columns

SELECT station,measurement,year FROM parquet.`../../Data/Weather_sampled.parquet` WHERE measurement="SNOW"
870 ['station', 'measurement', 'year']


For more on DataFrames see: 
* [API for the DataFrame class](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame)

For more on Spark SQL see:
* [API for the pyspark.sql module](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module)